# Wall Force Visualization and Validation

In [ ]:
# Import Modules
import numpy as np
import numpy.linalg as la

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

from cosserat import Cosserat
from bspline import test_bspline, snake_bspline

In [ ]:
%load_ext autoreload
%autoreload 2

### Contact Force (Wall Force)

In [ ]:
# Function needs n = num_of_elements, f_element, current_r, and current_v
# f_element will be constant throughout out simulations as we just have gravity so we can calc that force beforehand
# this will save time since we won't have to calc that force every time the wall_force function is run
def wall_force (n, f_gravity, r, v):
    # Extract the y-components of gravity for each element
    f_element = f_gravity [:, 1]
    # wall stiffness and dissipation constants
    k_w = 1
    gamma_w = 1e-6
    # Normal vector by the wall
    wall_normal = np.zeros((n, 3, )) # (n,3, )
    wall_normal[:] = [0., 1., 0.]

    # Penetration distance: epsilon
    y_coord = r[:, 1]  # A vector of y - coordinates of each node: dimensions: (n+1, )
    epsilon = (y_coord [:-1] + y_coord [1:]) / 2. # Penetration depth of each element's centre: (n,)

    #Velocity of the centre of each element (in y-direction only)
    velo_element = (v [1:,1] + v [:-1,1])/2. # (n, )

    # wall_force on each element: ((n, 3, ))
    f_wall_elem = np.zeros((n, 3, ))
    # updating just the y-component of wall force as no force is exerted along x and z axis
    
    f_wall_elem [:, 1] = np.heaviside(- epsilon, 1.) * (- f_element - k_w * epsilon - gamma_w * velo_element)
    # wall_force on each node: ((n+1, 3, ))
    f_wall_nodes = np.zeros((n+1, 3,))
    f_wall_nodes[:-1] += (f_wall_elem)/2.
    f_wall_nodes[1:] += (f_wall_elem)/2.

    return f_wall_nodes, f_wall_elem

# Wall Force Validation:

In [ ]:
from setup_wallforce import setup, L, n, M

# Gravitational force on each element (only the y-component)
f_gravity_elements = np.zeros((n, 3, )) # (n,3,)
f_gravity_elements [:,1] = - (M/n) * 9.81

# Gravitational Force Vector on each node (will be used later for calculation of total_force)
f_gravity_nodes = np.zeros((n+1, 3, ))
f_gravity_nodes [:-1] += f_gravity_elements/2.
f_gravity_nodes [1:] += f_gravity_elements/2.

n_step = 10000 # number of steps (3 secs)
dt = 3e-4

solution = Cosserat(**setup) # Should initialize the variables and setup the boundary conditions in cosserat.py code
state = solution.get_state # Initial state
y_list = [] # Height list
v_list = [] # Velocity (y-direction)
f_wall_force_list = []
f_total_force_list = []

r, v, Q, w = state[0], state[1], state[2], state[3]
couple = np.zeros((n,3,))

for itr in range(n_step):

#     shear_nodes[0], shear_nodes[-1]  = 2 * shear_nodes[0], 2 * shear_nodes[-1]  
#     # Calc shear at the elements: [n]
#     shear_elements = (shear_nodes [:-1] + shear_nodes[1:])/2.

    # (i) Wall Force Calculation:
    f_wall_nodes, f_wall_elements = wall_force (n, f_gravity_elements, r, v)
    f_wall_force_list.append(f_wall_nodes[1][1])
    
#     # (ii) Friction Force Calculation:
#     F = f_wall_elements + f_gravity_elements + shear_elements # Total force on each element
#     f_friction_nodes = friction_long (v, tangent, F, f_wall_elements, n, mu_sf, mu_kf, mu_sb, mu_kb)

#     # (iii) Total Force Calculation at each node:
    f_total_nodes = f_wall_nodes + f_gravity_nodes #+ f_friction_nodes
    f_total_force_list.append(f_total_nodes[1][1])

#     # (iv) Torque Calculation:
#     couple = torque(s_elements, beta_m, Q, time, lambda_m) # I guess you want the couple at the elements ie shape = [n]

    r, v, Q, w = solution.step(dt = dt, couple = couple, force = f_total_nodes)
    
    y_list.append(r[:,1])
    v_list.append(v[:,1])

In [ ]:
plt.plot(f_wall_force_list)
plt.ylabel("CASE 3: Wall Force on element - 1")
plt.xlabel("No. of Iterations")
plt.show()

plt.plot(f_total_force_list)
plt.ylabel("CASE 3: Total Force on element - 1")
plt.xlabel("No. of Iterations")
plt.show()

In [ ]:
plt.plot(y_list)
plt.ylabel("CASE 3: Y-Position")
plt.xlabel("No. of Iterations")
plt.show()

plt.plot(v_list)
plt.ylabel("CASE 3: Y-Direction Velocity")
plt.xlabel("No. of Iterations")
plt.show()

In [ ]:
# CASE 1:
# --> Initial Penetration of 'r' and no gravity

In [ ]:
# CASE 2:
# --> Initial Penetration of 'r' and gravity ON

In [ ]:
# CASE 3:
# --> Initial Height of 'r' and gravity ON